In [50]:
import pandas as pd    
import numpy as np 

import pyecharts.options as opts
from pyecharts.globals import ThemeType
from pyecharts.charts import Line,Bar,Map,Pie,Grid,Tab,Timeline
 
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB 

df_confirmed = pd.read_csv('data/world_confirmed.csv',
                           header=0, index_col=[1, 0]).drop(['Lat', 'Long'], axis=1)
df_death = pd.read_csv('data/world_death.csv',
                       header=0, index_col=[1, 0]).drop(['Lat', 'Long'], axis=1)
df_recovery = pd.read_csv('data/world_recovery.csv',
                          header=0, index_col=[1, 0]).drop(['Lat', 'Long'], axis=1)

sum_confirmed = df_confirmed.groupby('Country/Region').sum()
sum_death = df_death.groupby('Country/Region').sum()
sum_recovery = df_recovery.groupby('Country/Region').sum()
 

In [51]:
name_map = {
    "US": "United States",
    "Korea, South":"Korea",
    "Congo (Brazzaville)":"Congo",
    "Congo (Kinshasa)":"Dem. Rep. Congo",
    "Czechia":"Czech Rep.",
    "North Macedonia":"Macedonia",
    "Bosnia and Herzegovina":"Bosnia and Herz.",
    "Laos":"Lao PDR",
    "Dominica":"Dominican Rep.",
    "Central African Republic":"Central African Rep."
}


def get_nation_name(name):
    if name in name_map:
        return name_map[name]
    return name

visual_opts = opts.VisualMapOpts(max_=100000, is_piecewise=True,
                                 pieces=[
                                     {"max": 100000, "min": 10001,
                                      "label": ">1000", "color": "#6A0C0C"},
                                     {"max": 10000, "min": 1001,
                                      "label": ">1000", "color": "#8A0808"},
                                     {"max": 1000, "min": 500,
                                      "label": "500-1000", "color": "#FF0000"},
                                     {"max": 499, "min": 100,
                                      "label": "100-499", "color": "#B8860B"},
                                     {"max": 99, "min": 10,
                                      "label": "10-99", "color": "#EEB422"},
                                     {"max": 9, "min": 1, "label": "1-9",
                                      "color": "#EEDD82"},
                                     {"max": 0, "min": 0, "label": "0",
                                      "color": "#FFFFFF"},
                                 ])
 
tl = Timeline()
for day in sum_confirmed.columns:  
    s = sum_confirmed[day]
    pairlst = [(get_nation_name( s.index[i]),int(s.values[i])) for i in range(len(s))]
    map0 = (
        Map(init_opts=opts.InitOpts(theme=ThemeType.DARK))
        .add("confirmed case",pairlst, "world",is_map_symbol_show=False,label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(visualmap_opts=visual_opts) 
    )
    tl.add(map0, day) 

tl.load_javascript()

In [52]:
tl.render_notebook()

In [53]:
total_confirmed = sum_confirmed[sum_confirmed.columns[-1]]
total_death = sum_death[sum_death.columns[-1]]
total_recovery = sum_recovery[sum_recovery.columns[-1]]

In [56]:
top_confirmed = total_confirmed.nlargest(20)
top_confirmed.set_value('others',total_confirmed.sum()-top_confirmed.sum())
top_death = total_death.nlargest(20)
top_death.set_value('others',total_death.sum()-top_death.sum())

# pie chart of confirmed cases
pie = Pie(init_opts=opts.InitOpts(theme=ThemeType.DARK))
pie_data = [(top_confirmed.index[i],int(top_confirmed.values[i])) for i in range(len(top_confirmed))]
pie.add('Confirmed Cases', data_pair=pie_data)
pie.set_global_opts(legend_opts=opts.LegendOpts(is_show=False)) 

# pie chart of death toll
pie_death = Pie(init_opts=opts.InitOpts(theme=ThemeType.DARK))
pie_death_data = [(top_death.index[i],int(top_death.values[i])) for i in range(len(top_death))]
pie_death.add('Fatality toll', data_pair=pie_death_data)
pie_death.set_global_opts(legend_opts=opts.LegendOpts(is_show=False))

# bar chart of confirmed cases
bar = Bar(init_opts=opts.InitOpts(theme=ThemeType.DARK))
bar_data = [(top_confirmed.index[i],int(top_confirmed.values[i])) for i in range(len(top_confirmed))] 
bar.add_xaxis(top_confirmed.index.tolist()).reversal_axis()
bar.add_yaxis('Confirmed Cases',top_confirmed.values.tolist(),label_opts=opts.LabelOpts(position="right"))  
bar.set_global_opts(legend_opts=opts.LegendOpts(is_show=False))

# bar chart of death toll
bar_death = Bar(init_opts=opts.InitOpts(theme=ThemeType.DARK))
bar_death_data = [(top_death.index[i],int(top_death.values[i])) for i in range(len(top_death))] 
bar_death.add_xaxis(top_death.index.tolist()).reversal_axis()
bar_death.add_yaxis('Fatality toll',top_death.values.tolist(),label_opts=opts.LabelOpts(position="right"))

bar_death.set_global_opts(legend_opts=opts.LegendOpts(is_show=False))

tab =  Tab() 
tab.add(pie,"Confirmed Cases")
tab.add(pie_death,"Fatality toll")
tab.add(bar,"Confirmed Cases")
tab.add(bar_death,"Fatality toll")

tab.load_javascript()

In [57]:
tab.render_notebook()